In [1]:
!wget -O AnyLengthBertModelAndUtils.py https://raw.githubusercontent.com/DominicJW/AnyLengthBert/main/AnyLengthBertModelAndUtils.py
!pip install datasets

#from google.colab import drive
# drive.mount('/content/drive')
# drive_path = '/content/drive/My Drive/'

#not using google drive
!wget -O train.csv https://drive.usercontent.google.com/download?id=1B6h9GBUoiWV00FHWWv4XL7OKWSv2peTR&export=download
!wget -O dev.csv https://drive.usercontent.google.com/download?id=1Rh6ZgNk6svFvlXokAnX2OeY7zw9ghXlH&export=download

drive_path = "./" #ease for switching between using drive and using wget

--2024-04-23 01:38:50--  https://raw.githubusercontent.com/DominicJW/AnyLengthBert/main/AnyLengthBertModelAndUtils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12727 (12K) [text/plain]
Saving to: ‘AnyLengthBertModelAndUtils.py’

AnyLengthBertModelA 100%[===================>]  12.43K  --.-KB/s    in 0s      

2024-04-23 01:38:50 (108 MB/s) - ‘AnyLengthBertModelAndUtils.py’ saved [12727/12727]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from AnyLengthBertModelAndUtils import SentencePairClassifier, my_collate, CustomDataset, set_seed, evaluate_loss
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup


import numpy as np
import pandas as pd
from datasets import load_dataset, load_metric
from  datasets import Dataset as datasetsDataset #Apologies, this was to resolve the nameclash with pytorch
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, matthews_corrcoef,accuracy_score

import os
import copy
import random
import math
import string


In [3]:
def train_bert(net, optimizer, lr_scheduler, train_loader, val_loader, epochs, iters_to_accumulate):
    best_loss = np.Inf
    best_ep = 1
    nb_iterations = len(train_loader)
    print_every = nb_iterations // 5
    iters = []
    train_losses = []
    val_losses = []
    scaler = GradScaler()
    for ep in range(epochs):
        net.train()
        running_loss = 0.0
        for batch_idx, batch in enumerate(tqdm(train_loader)):
            seq = batch[0].to(device)
            attn_masks = batch[1].to(device)
            labels = batch[2].to(device)
            num_chunk = batch[3].to(device)
            with autocast():
                logits = net(seq, attn_masks,num_chunk)
                loss = criterion(logits.squeeze(-1), labels.float())
                loss = loss / iters_to_accumulate

            scaler.scale(loss).backward()
            if (batch_idx + 1) % iters_to_accumulate == 0 or (batch_idx + 1) == len(train_loader):
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(net.parameters(), gradient_clip_val)
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                lr_scheduler.step()


            running_loss += loss.item()

            if (batch_idx+ 1) % print_every == 0:  # Print training loss information
                print()
                print("Iteration {}/{} of epoch {} complete. Loss : {} "
                    .format(batch_idx+1, nb_iterations, ep+1, running_loss/print_every))

                running_loss = 0.0



        val_loss,val_acc,val_prec,val_rec,val_f1,val_mcc = evaluate_loss(net, val_loader)  # Compute validation loss
        print("Epoch : {} \n Loss : {} Accuracy : {} \n Precision : {} Recall : {} \n f1 : {} mcc : {} ".format(ep+1,val_loss,val_acc,val_prec,val_rec,val_f1,val_mcc))



In [4]:
delimiter = ","
df = pd.read_csv( os.path.join(drive_path, "train.csv"), delimiter=delimiter)
cols = list(df.columns)
df.columns = cols
dataset = datasetsDataset.from_pandas(df)
subdataset = dataset
split = subdataset.train_test_split(test_size=0.1, seed=1)
train = split['train']
val = split['test']
test =  pd.read_csv(os.path.join(drive_path,"dev.csv"))
test.columns = cols
df_train = pd.DataFrame(train)
df_val = pd.DataFrame(val)
df_test = pd.DataFrame(test)

In [5]:
maxlen = 128
criterion = nn.BCEWithLogitsLoss()

bert_model = "bert-base-uncased"
train_set = CustomDataset(df_train,bert_model,maxlen)
val_set = CustomDataset(df_val, bert_model,maxlen)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [6]:
torch.cuda.empty_cache()
set_seed(1)
freeze_bert = False
lr = 2e-5
epochs = 2
gradient_clip_val = 5.0
bs = 32
train_loader = DataLoader(train_set, batch_size=bs, shuffle=True,collate_fn=my_collate)
val_loader = DataLoader(val_set, batch_size=bs, shuffle = False,collate_fn=my_collate)



In [8]:
net = SentencePairClassifier(bert_model,freeze_bert=freeze_bert)
net.to(device)
net.train()
optimizer = AdamW(net.parameters(), lr=lr)
iters_to_accumulate = 1 ##Effective batch size = batch size
num_warmup_steps = 500
num_training_steps = len(train_loader) * 4 #This is to make the last learn rate to be 0.5x max, instead of 0
lr_scheduler = get_linear_schedule_with_warmup(optimizer=optimizer, num_warmup_steps=500, num_training_steps=num_training_steps)

train_bert(net,optimizer,lr_scheduler,train_loader,val_loader,epochs,iters_to_accumulate=iters_to_accumulate)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
 20%|██        | 134/667 [00:17<01:06,  8.00it/s]


Iteration 133/667 of epoch 1 complete. Loss : 0.646376136326252 


 40%|████      | 267/667 [00:35<01:04,  6.23it/s]


Iteration 266/667 of epoch 1 complete. Loss : 0.45923641693771333 


 60%|█████▉    | 400/667 [00:53<00:37,  7.17it/s]

...


Iteration 532/667 of epoch 2 complete. Loss : 0.26735147609746546 


100%|█████████▉| 666/667 [01:29<00:00,  8.02it/s]


Iteration 665/667 of epoch 2 complete. Loss : 0.2608935890117086 


100%|██████████| 667/667 [01:29<00:00,  7.44it/s]


Epoch : 2 
 Loss : 0.30556083778540294 Accuracy : 0.8705187684521299 
 Precision : 0.7413793103448276 Recall : 0.80248833592535 
 f1 : 0.7707244212098582 mcc : 0.6817432531591121 


In [12]:
torch.save(net,os.path.join(drive_path,"AnyLengthBert3.pt"))